In [113]:
import pandas as pd
from sklearn.metrics import accuracy_score

df = pd.read_csv("dataset.csv", header=None, names=["artist", "song", "genre", "tokens"])

def to_list(x):
    return x[1:-1].split(',')


df['tokens'] = df['tokens'].apply(to_list)

In [145]:
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer

data = df.values

def process_y(d):
    for i in range(len(d)):
        if d[i] != 'hiphop': d[i] = 'other'
    return d

def dummy_fun(doc):
    return doc

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)  

X = tfidf.fit_transform(data[:,3])
X_train = X[:5000]
X_test = X[5000:]

Y = process_y(data[:, 2])
Y_train = Y[:5000]
Y_test = Y[5000:]

In [146]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB().fit(X_train, Y_train)

predicted = clf.predict(X_test)
accuracy_score(predicted, Y_test)

0.9077404222048475

In [147]:
from sklearn.linear_model import SGDClassifier

svm = SGDClassifier().fit(X_train, Y_train)

predicted = clf.predict(X_test)
accuracy_score(predicted, Y_test)


/Users/IzKevin/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.9077404222048475

In [148]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf.fit(X_train, Y_train)

predicted = clf.predict(X_test)
accuracy_score(predicted, Y_test)

0.8717748240813136

In [ ]:
from numpy import genfromtxt

data = genfromtxt('lyrics.csv', delimiter=',')
